# Followers scraping

The idea is to scrape the ammount of followers in Instagram and Facebook of every actor.

### 1. Import the libraries

In [1]:
import numpy as np
import pandas as pd

### 2. Import the dataset

In [3]:
df = pd.read_excel("final_netflix_df.xlsx")
df.head()

,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,title,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors
0,Enter the Anime,Documentary,2019-08-05,58,2.5,English/Japanese,2019,616904,tt10662450,What is anime? Through deep-dives with notable...,...,Enter the Anime,Enter the Anime,Documentary,?,United States of America,Japan,"['Kouzou Morishita', 'Kenji Kamiyama', 'Youko ...",['Alex Burunova'],4,2
1,Dark Forces,Thriller,2020-08-21,81,2.6,Spanish,2020,735110,tt12536776,"In search of his sister, a renegade criminal s...",...,Dark Forces,Fuego negro,Horror,Action,?,?,"['Tenoch Huerta', 'Eréndira Ibarra', 'Ariane P...",['Bernardo Arellano'],2,2
2,The App,Science fiction/Drama,2019-12-26,79,2.6,Italian,2019,653522,tt11385066,"Loving girlfriend, family fortune, breakout mo...",...,The App,The App,Drama,Science Fiction,Italy,?,"['Vincenzo Crea', 'Greta Scarano', 'Maya Sansa...",['Elisa Fuksas'],3,3
3,Sex: Unzipped,Comedy,2021-10-26,59,3.1,English,2021,890280,tt15523050,Rap superstar Saweetie hosts a celebration of ...,...,Sex: Unzipped,Sex: Unzipped,Comedy,?,United States of America,?,"['Saweetie', 'Joel Kim Booster', 'Michelle But...",[],3,7
4,Swallow,Drama,2021-10-01,128,3.2,English,2021,874562,tt14391622,"Set in the 1980s, Tolani Ajao is a bank secret...",...,Swallow,Swallow,Drama,History,?,?,[],['Kunle Afolayan'],0,0


### 3. Build "main_cast" list

In [86]:
# Build "main_cast" with the first five actors of the entire cast
def main_cast(cast_column):
    '''Takes the cast_column and returns a main cast list with the first 5 actors if possible'''
    main_cast = cast_column.tolist()
    main_cast_list = []

    for cast in main_cast:
        actors = cast.replace("'", "").replace("[", "").replace("]", "").split(",")
        actors_clean = []
        for actor in actors:
            actors_clean.append(actor.strip())
        main_cast_list.append(actors_clean)
    
    main_cast_final = []

    for cast in main_cast_list:
        if len(cast) < 5:
            main_cast_final.append(cast)
        else:
            main_cast_final.append(cast[0:5])
        

    
    return main_cast_final

In [87]:
main_cast = main_cast(df["cast"])
main_cast

[['Kouzou Morishita',
  'Kenji Kamiyama',
  'Youko Takahashi',
  'Alex Burunova',
  'Adi Shankar'],
 ['Tenoch Huerta', 'Eréndira Ibarra', 'Ariane Pellicer', 'Nick Zedd'],
 ['Vincenzo Crea',
  'Greta Scarano',
  'Maya Sansa',
  'Abel Ferrara',
  'Anita Kravos'],
 ['Saweetie',
  'Joel Kim Booster',
  'Michelle Buteau',
  'Nikki Glaser',
  'Dominique Jackson'],
 [''],
 ['Dylan Minnette',
  'Piercey Dalton',
  'Sharif Atkins',
  'Patricia Bethune',
  'Matt Angel'],
 ['Shabana Azmi', 'Sanjeeda Sheikh'],
 ['Sushant Singh Rajput',
  'Jacqueline Fernandez',
  'Boman Irani',
  'Pankaj Tripathi',
  'Sapna Pabbi'],
 ['Amanda Rawles',
  'Maizura',
  'Asmara Abigail',
  'Ayushita Nugraha',
  'Joko Anwar'],
 ['Demet Akbağ', 'Haluk Bilginer', 'Elçin Sangu', 'Alican Yücesoy'],
 ['Edgar Ramírez',
  'Michael Pitt',
  'Anna Brewster',
  'Sharlto Copley',
  'Tamer Burjaq'],
 ['Neil Young',
  'Willie Nelson',
  'Hilary Shepard',
  'Page Hannah',
  'Hayley DuMond'],
 ['Giovana Cordeiro',
  'Gessica Kayane',

In [88]:
print(len(main_cast), df.shape[0])

684 684


In [89]:
# Number of actors to scrape
actors_to_scrape = 0

for actors in main_cast:
    actors_to_scrape += len(actors)
    
actors_to_scrape

2764

The list is already prepared

### 4. Scraping

Most of the actors won't have their instagram user with their real names but with an acronym or a nickname. We have to create some functions to automate Google's search and get the number of followers. The first search offered by Google will be the actual Instagram page of every actor.


- The code is based on this site:
https://practicaldatascience.co.uk/data-science/how-to-scrape-google-search-results-using-python

In [41]:
# pip install requests_html

In [42]:
# Import the libraries
import requests
import urllib
from requests_html import HTML
from requests_html import HTMLSession

In [43]:
# First function: to pass our URL to Requests-HTML and return the source code of the page.
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """
    
    session = HTMLSession()
    # Make a response object with the URL
    response = session.get(url)
    return response

In [52]:
# Second function: to format and URL encode the query, send it to Google and show the output.
def get_results(query):
    
    # Safe mode for quoting the URL replacing special characters
    query = urllib.parse.quote_plus(query)
    # Once the query is "safe" apply the get_source function
    response = get_source("https://www.google.com/search?q=" + query + " instagram")
    
    return response

In [53]:
# Third function: to get results
def parse_results(response):

    # Select elements with CSS selectors; the CSS can change in the future
    css_identifier_title = "h3"
    css_identifier_result = ".tF2Cxc"
    css_identifier_text = ".IsZvec"
    
    results = response.html.find(css_identifier_result)
    
    output = []
    
    for result in results:

        try:
            item = {'text': result.find(css_identifier_text, first=True).text,
                   'title': result.find(css_identifier_title, first=True).text}
        
            output.append(item)
        
        except:
            output.append("")
        
    return output[0]

In [54]:
# Fourth function: to print result
def google_search(query):
    response = get_results(query)
    
    try:
        result = parse_results(response)
    
        return result
    
    except:
        
        return ""

In [57]:
# Prove:
results = google_search("Alex burunova")
results

{'text': '2282 Followers, 933 Following, 575 Posts - See Instagram photos and videos from Alex Burunova (@alexburunova)',
 'title': 'Alex Burunova (@alexburunova) • Instagram photos and videos'}

### 5. Scrape followers

For computational reasons we have to scrape by parts

In [58]:
actors_1 = main_cast[0:50]

In [61]:
followers = []

for cast in actors_1:
    followers_x = []
    for actor in cast:
        result = google_search(actor)
        followers_x.append(result["text"])
    followers.append(followers_x)

In [62]:
followers

[['Kozo Morishita. ビニールハウス専門店モリシタの代表です。弊社は今年で創業52年目です。これからも福井県の施設園芸の発展に貢献します。',
  '206 Followers, 183 Following, 142 Posts - See Instagram photos and videos from Kenji Kamiyama (@kamiyama_kenji)',
  '6495 Followers, 21 Following, 459 Posts - See Instagram photos and videos from 高橋洋子 Yoko TAKAHASHI (@yoko_takahashi_official)',
  '2282 Followers, 933 Following, 575 Posts - See Instagram photos and videos from Alex Burunova (@alexburunova)',
  '12.6k Followers, 1958 Following, 294 Posts - See Instagram photos and videos from Adi Shankar (@bootleguniverse)'],
 ['106k Followers, 507 Following, 343 Posts - See Instagram photos and videos from Tenoch Huerta (@tenochhuerta)',
  '455k Followers, 5343 Following, 1823 Posts - See Instagram photos and videos from @erendiritas.',
  '2581 Followers, 503 Following, 236 Posts - See Instagram photos and videos from Ariane Pellicer (@ariane_pellicer)',
  '10 mar 2020 — 64 Likes, 3 Comments - Nick Zedd (@nickzedd) on Instagram: “#nickzedd #cultfilms #ava

In [72]:
# Cleaning
followers_clean = []

for cast in followers:
    clean = []
    for actor in cast:
        text = actor.split(" ")
        clean.append(text[0])
    followers_clean.append(clean)

followers_clean

[['Kozo', '206', '6495', '2282', '12.6k'],
 ['106k', '455k', '2581', '10'],
 ['3939', '153k', '8302', '175', '1314'],
 ['12.7m', '101k', '236k', '803k', '870k'],
 ['Create'],
 ['7.3m', '2459', '460', '2661', '180'],
 ['149k', '4.4m'],
 ['13.2m', '56.3m', '796k', '3.8m', '617k'],
 ['2.8m', '193k', '70.5k', '1.1m', '288k'],
 ['969k', '224k', '8.1m', '150k'],
 ['3.8m', '4333', '45.4k', '17.3k', '1074'],
 ['165k', '771k', '45.6k', '58k', '403'],
 ['1.1m', '15.1m', '185k', 'Agenciamento', '18.4m'],
 ['975', '2m', '48.5k', '22.7k', '231'],
 ['12.4m', '861k', '17.9m', '2090', '11.2k'],
 ['4622', '11.1m'],
 ['12.1m', '4272', '192k', 'Related', '107k'],
 ['3.7m', '2', '16.1k', '1m'],
 ['3.6m', '934k'],
 ['1.9m', '52.4k', '161', '576', '2.2m'],
 ['81.8k', 'Aidan', '529', '179', '361'],
 ['5m', '6.2m', '104k', '115k', '253k'],
 ['2893', '19.3k', '161k', '29', '4083'],
 ['3676'],
 ['40.1m', '3.5m', '4.2m', '293k', '2.1m'],
 ['Create'],
 ['13.9m', '767k', '599k', '111k', '2.9m'],
 ['21m', '21.3k', 

In [74]:
# Build followers column
df["followers"] = None
df["followers"]

0      None
1      None
2      None
3      None
4      None
       ... 
679    None
680    None
681    None
682    None
683    None
Name: followers, Length: 684, dtype: object

In [75]:
df.iloc[0:50, -1] = followers_clean
df.head(10)

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
0,Enter the Anime,Documentary,2019-08-05,58,2.5,English/Japanese,2019,616904,tt10662450,What is anime? Through deep-dives with notable...,...,Enter the Anime,Documentary,?,United States of America,Japan,"['Kouzou Morishita', 'Kenji Kamiyama', 'Youko ...",['Alex Burunova'],4,2,"[Kozo, 206, 6495, 2282, 12.6k]"
1,Dark Forces,Thriller,2020-08-21,81,2.6,Spanish,2020,735110,tt12536776,"In search of his sister, a renegade criminal s...",...,Fuego negro,Horror,Action,?,?,"['Tenoch Huerta', 'Eréndira Ibarra', 'Ariane P...",['Bernardo Arellano'],2,2,"[106k, 455k, 2581, 10]"
2,The App,Science fiction/Drama,2019-12-26,79,2.6,Italian,2019,653522,tt11385066,"Loving girlfriend, family fortune, breakout mo...",...,The App,Drama,Science Fiction,Italy,?,"['Vincenzo Crea', 'Greta Scarano', 'Maya Sansa...",['Elisa Fuksas'],3,3,"[3939, 153k, 8302, 175, 1314]"
3,Sex: Unzipped,Comedy,2021-10-26,59,3.1,English,2021,890280,tt15523050,Rap superstar Saweetie hosts a celebration of ...,...,Sex: Unzipped,Comedy,?,United States of America,?,"['Saweetie', 'Joel Kim Booster', 'Michelle But...",[],3,7,"[12.7m, 101k, 236k, 803k, 870k]"
4,Swallow,Drama,2021-10-01,128,3.2,English,2021,874562,tt14391622,"Set in the 1980s, Tolani Ajao is a bank secret...",...,Swallow,Drama,History,?,?,[],['Kunle Afolayan'],0,0,[Create]
5,The Open House,Horror thriller,2018-01-19,94,3.2,English,2018,485774,tt7608028,A teenager and his mother find themselves besi...,...,The Open House,Horror,Thriller,United States of America,?,"['Dylan Minnette', 'Piercey Dalton', 'Sharif A...","['Matt Angel', 'Suzanne Coote']",7,4,"[7.3m, 2459, 460, 2661, 180]"
6,Kaali Khuhi,Mystery,2020-10-30,90,3.4,Hindi,2020,744876,tt10235600,When a restless spirit curses a Punjab village...,...,काली खुहि,Horror,Drama,India,?,"['Shabana Azmi', 'Sanjeeda Sheikh']",['Terrie Samundra'],0,2,"[149k, 4.4m]"
7,Drive,Action,2019-11-01,147,3.5,Hindi,2019,466550,tt6593054,A getaway driver is attracted to a female neig...,...,Drive,Action,Thriller,India,?,"['Sushant Singh Rajput', 'Jacqueline Fernandez...",['Tarun Mansukhani'],3,2,"[13.2m, 56.3m, 796k, 3.8m, 617k]"
8,A World Without,Science fiction / teen drama,2021-10-14,107,3.7,Indonesian,2021,877372,tt15484488,"Founded by the power couple, Ali and Sofia Kha...",...,A World Without,Mystery,Thriller,?,?,"['Amanda Rawles', 'Maizura', 'Asmara Abigail',...",['Nia Dinata'],3,6,"[2.8m, 193k, 70.5k, 1.1m, 288k]"
9,Leyla Everlasting,Comedy,2020-12-04,112,3.7,Turkish,2020,668156,tt11792734,"A resilient housewife, her husband and their m...",...,9 Kere Leyla,Comedy,?,Turkey,?,"['Demet Akbağ', 'Haluk Bilginer', 'Elçin Sangu...",['Ezel Akay'],2,2,"[969k, 224k, 8.1m, 150k]"


#### 5.1 Defining the functions

In [107]:
def clean_followers(result):
    ''' Takes the list of the scraped results and it gets the followers
    
    Args:
        result (list): list of scraped results
        
    Returns:
        followers_clean (list): list with the followers'''
    
    followers_clean = []

    for cast in result:
        clean = []
        for actor in cast:
            text = actor.split(" ")
            clean.append(text[0])
        followers_clean.append(clean)

    return followers_clean

In [108]:
def scrape_followers(actors_list):
    '''Takes a Pandas series in input with actors and scrape their respective followers
    
    Args:
        actors_list (pd.Series): Series with the actors to be scraped
        
    Returns:
        followers (list): list with the scraped results
    '''
    followers = []

    for cast in actors_list:
        followers_x = []
        for actor in cast:
            result = google_search(actor)
            followers_x.append(result["text"])
        followers.append(followers_x)
    
    return clean_followers(followers)

In [100]:
df.iloc[96, :]

Title                                                  #REALITYHIGH
Genre                                                        Comedy
Premiere                                        2017-09-08 00:00:00
Runtime                                                          99
IMDB Score                                                      5.2
Language                                                    English
premiere_year                                                  2017
tmdb_id                                                      455656
imdb_id                                                   tt6119504
overview          When nerdy high schooler Dani finally attracts...
vote_average                                                    6.4
vote_count                                                      935
title                                                  #realityhigh
original_title                                         #realityhigh
genre_1                                         

###### Next steps

In [109]:
df.iloc[50:100, -1] = scrape_followers(main_cast[50:100])
df.iloc[90:100, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
90,See You Yesterday,Science fiction,2019-05-17,87,5.2,English,2019,582607,tt8743064,As two teen prodigies try to master the art of...,...,See You Yesterday,Science Fiction,Drama,United States of America,?,"['Eden Duncan-Smith', 'Dante Crichlow', 'Astro...",['Stefon Bristol'],11,5,"[9904, Danté, 별을, 14.1k, 4]"
91,Sandy Wexler,Comedy,2017-04-14,131,5.2,English,2017,419700,tt5893332,When a hapless but dedicated talent manager si...,...,Sandy Wexler,Comedy,?,United States of America,?,"['Adam Sandler', 'Kevin James', 'Rob Schneider...",['Steven Brill'],53,15,"[11.8m, 1.7m, 1.4m, 8.3m, 996k]"
92,Father of the Year,Comedy,2018-07-20,94,5.2,English,2018,531949,tt7256866,"Two college grads return to their hometown, wh...",...,Father of the Year,Comedy,?,United States of America,?,"['David Spade', 'Nat Faxon', 'Joey Bragg', 'Ma...",['Tyler Spindel'],10,6,"[2.5m, 461, 1.6m, 113k, 1.6m]"
93,Rim of the World,Science fiction adventure,2019-05-24,98,5.2,English,2019,531306,tt8179388,Stranded at a summer camp when aliens attack t...,...,Rim of the World,Science Fiction,Adventure,United States of America,?,"['Jack Gore', 'Miya Cech', 'Benjamin Flores Jr...",['McG'],20,8,"[65.1k, 463k, 1.2m, 265k, 21.4m]"
94,Monster Hunter: Legends of the Guild,CG animated fantasy,2021-08-12,58,5.2,English,2021,851281,tt9174358,In a world where humans and fearsome monsters ...,...,Monster Hunter: Legends of the Guild,Animation,Adventure,United States of America,?,"['Dante Basco', 'Brando Eaton', 'Erica Lindbec...",['Steve Yamamoto'],4,4,"[182k, 35.6k, 31.5k, 349k, 2270]"
95,Hubie Halloween,Comedy,2020-10-07,103,5.2,English,2020,617505,tt10682266,"Hubie Dubois, despite his devotion to his home...",...,Hubie Halloween,Comedy,Mystery,United States of America,?,"['Adam Sandler', 'Kevin James', 'Julie Bowen',...",['Steven Brill'],39,35,"[11.8m, 1.7m, 1.5m, 131k, 1.4m]"
96,#REALITYHIGH,Comedy,2017-09-08,99,5.2,English,2017,455656,tt6119504,When nerdy high schooler Dani finally attracts...,...,#realityhigh,Comedy,?,United States of America,?,"['Nesta Cooper', 'Keith Powers', 'Alicia Sanz'...",['Fernando Lebrija'],11,10,"[103k, 1.8m, 118k, 914k, 1.1m]"
97,Girlfriend's Day,Comedy,2017-02-14,70,5.2,English,2017,346650,tt2962984,"When he's caught up in a deadly conspiracy, an...",...,Girlfriend's Day,Comedy,Drama,United States of America,?,"['Bob Odenkirk', 'Amber Tamblyn', 'Stacy Keach...",['Michael Stephenson'],19,9,"[263k, 330k, 6908, 9779, 118]"
98,Still Laugh-In: The Stars Celebrate,Variety show,2019-05-14,60,5.2,English,2019,602310,tt9880234,"Hollywood's finest pay tribute to ""Rowan and M...",...,Still Laugh-In: The Stars Celebrate,Comedy,Music,?,?,"['Tiffany Haddish', 'Michael Douglas', 'Lily T...",[],17,15,"[6.8m, 972k, 3689, 65.1m, 8.6m]"
99,Hypnotic,Thriller,2021-10-27,88,5.2,English,2021,864873,tt12383460,A young woman seeking self-improvement enlists...,...,Hypnotic,Thriller,Drama,Canada,United States of America,"['Kate Siegel', ""Jason O'Mara"", 'Dulé Hill', '...","['Matt Angel', 'Suzanne Coote']",5,2,"[446k, 501, 378k, 9402, 94.9k]"


In [111]:
df.iloc[100:150, -1] = scrape_followers(main_cast[100:150])
df.iloc[140:150, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
140,Freaks: You're One of Us,Supernatural drama,2020-09-02,92,5.4,German,2020,658777,tt12875782,After having a chance encounter with a mysteri...,...,Freaks – Du bist eine von uns,Drama,Fantasy,Germany,?,"['Cornelia Gröschel', 'Tim Oliver Schultz', 'W...",['Felix Binder'],14,8,"[17.9k, 211k, 111k, 977, 2225]"
141,The Secret Diary of an Exchange Student,Romantic comedy,2021-08-18,96,5.4,Portuguese,2021,760154,tt11594766,Two best friends embark on a life-changing adv...,...,The Secret Diary of an Exchange Student,Comedy,Romance,Brazil,?,"['Larissa Manoela', 'Thati Lopes', 'Bruno Mont...",['Bruno Garotti'],2,6,"[41.6m, 392k, 842k, 627, 110k]"
142,The 8th Night,Thriller film,2021-07-02,115,5.4,Korean,2021,604360,tt14781176,With prayer beads in one hand and an ax in the...,...,제8일의 밤,Mystery,Thriller,South Korea,?,"['Lee Sung-min', 'Park Hae-jun', 'Kim You-jung...",['Kim Tae-hyoung'],7,3,"[588k, 10.9k, 5.1m, 160k, 384]"
143,Guilty,Thriller,2020-03-06,119,5.4,Hindi,2020,675592,tt10062614,A young woman from a small town accuses the co...,...,Guilty,Thriller,Drama,India,?,"['Kiara Advani', 'Akansha Ranjan Kapoor', 'Gop...",['Ruchi Narain'],2,2,"[21m, Show, 56.3k, 6784]"
144,Bright: Samurai Soul,Anime/fantasy,2021-10-12,80,5.5,Japanese,2021,839100,tt14837960,"Izo, a Ronin, and Raiden, an orc, work to brin...",...,ブライト: サムライソウル,Animation,Action,Japan,?,"['Yuuki Nomura', 'Daisuke Hirakawa', 'Shion Wa...",['Kyohei Ishiguro'],7,2,"[433, 55, Shion., 720k, 740]"
145,Been So Long,Musical,2018-10-26,100,5.5,English,2018,462469,tt1682886,A single mother in London's Camden Town hears ...,...,Been So Long,Drama,Music,United Kingdom,?,"['Michaela Coel', 'Arinzé Kene', 'Ronke Adekol...",['Tinge Krishnan'],6,6,"[100k, 17, 265k, 19.4k, 450k]"
146,A Very Murray Christmas,Comedy / Musical,2015-12-04,56,5.5,English,2015,364067,tt4537842,Bill Murray worries no one will show up to his...,...,A Very Murray Christmas,Comedy,Music,United States of America,?,"['Bill Murray', 'Paul Shaffer', 'George Cloone...",['Sofia Coppola'],8,6,"[189k, 10.3k, 34.1k, 151m, 30.1k]"
147,The Package,Black comedy,2018-08-10,94,5.5,English,2018,503619,tt7525778,When a group of teenagers goes on a spring bre...,...,The Package,Comedy,?,United States of America,?,"['Daniel Doheny', 'Geraldine Viswanathan', 'Lu...",['Jake Szymanski'],10,8,"[62.3k, 181k, 4532, 65.7k, 34.2k]"
148,Ankahi Kahaniya,Anthology film,2021-09-17,110,5.5,Hindi,2021,862701,tt15250998,"As big city life buzzes around them, lonely so...",...,अनकही कहानियाँ,Drama,Romance,India,?,"['Kunal Kapoor', 'Zoya Hussain', 'Abhishek Ban...","['Saket Chaudhary', 'Abhishek Chaubey', 'Ashwi...",3,3,"[296k, 62.4k, 487k, 494k, 749k]"
149,The Legacy of a Whitetail Deer Hunter,Adventure/Comedy,2018-07-06,83,5.5,English,2018,311307,tt4762824,A star of hunting videos strives to bond with ...,...,The Legacy of a Whitetail Deer Hunter,Comedy,?,United States of America,?,"['Josh Brolin', 'Danny McBride', 'Scoot McNair...",['Jody Hill'],5,1,"[3m, 348, 6404, 187k, 10.4k]"


In [114]:
df.iloc[150:200, -1] = scrape_followers(main_cast[150:200])
df.iloc[190:200, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
190,The Do-Over,Action comedy,2016-05-27,108,5.7,English,2016,389053,tt4769836,The life of a bank manager is turned upside do...,...,The Do-Over,Action,Adventure,United States of America,?,"['Adam Sandler', 'David Spade', 'Paula Patton'...",['Steven Brill'],19,13,"[11.8m, 2.5m, 403k, 16.4k, 996k]"
191,The Holiday Calendar,Romantic comedy,2018-11-02,95,5.7,English,2018,555850,tt8262802,A talented photographer stuck in a dead-end jo...,...,The Holiday Calendar,Romance,Comedy,United States of America,?,"['Kat Graham', 'Quincy Brown', 'Ethan Peck', '...",['Bradley Walsh'],7,4,"[7m, 4.4m, 63.1k, 62.6k, 573]"
192,Resort to Love,Romantic comedy,2021-07-29,101,5.7,English,2021,785539,tt12929990,Aspiring pop star Erica ends up as the enterta...,...,Resort to Love,Romance,Comedy,United States of America,?,"['Christina Milian', 'Jay Pharoah', 'Sinqua Wa...",['Steven K. Tsuchida'],4,6,"[6.8m, 606k, 282k, 20.1k, 42.5k]"
193,Velvet Buzzsaw,Thriller,2019-02-01,112,5.7,English,2019,463684,tt7043012,Big money artists and mega-collectors pay a hi...,...,Velvet Buzzsaw,Thriller,Mystery,United States of America,?,"['Jake Gyllenhaal', 'Rene Russo', 'Zawe Ashton...",['Dan Gilroy'],43,27,"[7m, 6941, 80.1k, 1643, 4095]"
194,Yes Day,Comedy,2021-03-12,86,5.7,English,2021,638597,tt8521876,A mom and dad who usually say no decide to say...,...,Yes Day,Comedy,Family,United States of America,?,"['Jennifer Garner', 'Edgar Ramírez', 'Jenna Or...",['Miguel Arteta'],17,14,"[11.5m, 3.8m, 6.4m, 4.8m, 461]"
195,A Classic Horror Story,Horror,2021-07-14,95,5.7,Italian,2021,768334,tt12877640,Five carpoolers traveling in a motorhome to a ...,...,A Classic Horror Story,Drama,Horror,Italy,?,"['Matilda Anna Ingrid Lutz', 'Francesco Russo'...","['Roberto De Feo', 'Paolo Strippoli']",4,4,"[174k, , 33.3k, 61.9k, 15.3k]"
196,One Take,Documentary,2020-06-18,85,5.7,Thai,2020,679063,tt12401232,Members of Thai girl group BNK48 share the ups...,...,BNK48: One Take,Documentary,?,Thailand,?,[],['Manatsanun Panlertwongskul'],0,0,[Create]
197,The Woman in the Window,Psychological thriller,2021-05-14,100,5.7,English,2021,520663,tt6111574,An agoraphobic woman living alone in New York ...,...,The Woman in the Window,Drama,Thriller,United States of America,?,"['Amy Adams', 'Gary Oldman', 'Fred Hechinger',...",['Joe Wright'],7,6,"[803k, 136k, 243k, 2.1m, 3624]"
198,Little Evil,Comedy horror,2017-09-01,94,5.7,English,2017,346671,tt2937366,"Gary, who has just married Samantha, the woman...",...,Little Evil,Comedy,Horror,United States of America,?,"['Adam Scott', 'Evangeline Lilly', 'Bridget Ev...",['Eli Craig'],10,5,"[865k, 2.1m, 167k, 728k, 1331]"
199,Point Blank,Action,2019-07-12,86,5.7,English,2019,531503,tt2499472,A nurse is forced to spring a wounded murder s...,...,Point Blank,Action,Thriller,United States of America,?,"['Anthony Mackie', 'Frank Grillo', 'Marcia Gay...",['Joe Lynch'],13,4,"[2.8m, 571k, 102k, 409k, 751]"


In [116]:
df.iloc[200:250, -1] = scrape_followers(main_cast[200:250])
df.iloc[240:250, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
240,Special Correspondents,Satire,2016-04-29,100,5.8,English,2016,355008,tt4181052,A radio journalist and his technician get in o...,...,Special Correspondents,Comedy,?,Canada,United Kingdom,"['Ricky Gervais', 'Eric Bana', 'Vera Farmiga',...",['Ricky Gervais'],13,10,"[3.2m, 17.5k, 1m, 2092, 251k]"
241,TAU,Science fiction/Thriller,2018-06-29,97,5.8,English,2018,411143,tt4357394,"Held captive in a futuristic smart house, a wo...",...,Tau,Science Fiction,Thriller,United States of America,?,"['Maika Monroe', 'Ed Skrein', 'Gary Oldman', ""...","[""Federico D'Alessandro""]",5,2,"[177k, 255k, 136k, 4, 2747]"
242,The After Party,Comedy,2018-08-24,89,5.8,English,2018,538604,tt3960240,When an aspiring rapper goes viral for the wro...,...,The After Party,Comedy,Music,United States of America,?,"['Shelley Hennig', 'Andy Buckley', 'Amin Josep...",['Ian Edelman'],10,3,"[3.3m, 110k, 117k, 231k, 13.6m]"
243,Rajma Chawal,Comedy-drama,2018-11-30,118,5.8,Hindi,2018,494950,tt6747420,A father attempts to reconnect with his estran...,...,Rajma Chawal,Comedy,Drama,India,?,"['Rishi Kapoor', 'Amyra Dastur', 'Nirmal Rishi...",['Leena Yadav'],4,2,"[1.5m, 2.8m, 550, 1.2m, 365]"
244,Army of the Dead,Zombie/Heist,2021-05-21,148,5.9,English,2021,503736,tt0993840,"Following a zombie outbreak in Las Vegas, a gr...",...,Army of the Dead,Action,Adventure,United States of America,?,"['Dave Bautista', 'Ella Purnell', 'Omari Hardw...",['Zack Snyder'],21,23,"[3.7m, 458k, 3.7m, 1.1m, 647k]"
245,My Amanda,Romance,2021-07-15,89,5.9,Filipino,2021,843808,tt13136084,Two unusually close friends share every aspect...,...,My Amanda,Romance,Drama,Philippines,?,"['Alessandra de Rossi', 'Luz Valdez', 'Helga K...",['Alessandra de Rossi'],0,3,"[1m, 37.9k, 198k]"
246,True Memoirs of an International Assassin,Action comedy,2016-11-11,98,5.9,English,2016,339396,tt1542768,After a publisher changes a writer's debut nov...,...,True Memoirs of an International Assassin,Comedy,Action,United States of America,?,"['Kevin James', 'Kim Coates', 'Maurice Compte'...",['Jeff Wadlow'],17,6,"[1.7m, 21.3k, 41.3k, 280k, 2350]"
247,Amateur,Sports-drama,2018-04-06,96,5.9,English,2018,515333,tt5580392,14-year-old basketball phenom Terron Forte has...,...,Amateur,Drama,?,United States of America,?,"['Michael Rainey Jr.', 'Josh Charles', 'Sharon...",['Ryan Koo'],5,1,"[1.8m, 54.4k, 167k, 227k, 3242]"
248,Sisters on Track,Documentary,2021-06-24,96,5.9,English,2021,816518,tt9337574,Three track star sisters face obstacles in lif...,...,Sisters on Track,Documentary,?,United States of America,?,[],"['Tone Grøttjord-Glenne', 'Corinne van der Bor...",0,0,[Create]
249,6 Balloons,Drama,2018-04-06,75,5.9,English,2018,500634,tt6142496,"Over the course of one night, a woman drives a...",...,6 Balloons,Drama,?,United States of America,?,"['Abbi Jacobson', 'Dave Franco', 'Tim Matheson...",['Marja-Lewis Ryan'],3,5,"[abbi, Dave, 22.5k, 14.2k, 180k]"


In [118]:
df.iloc[250:300, -1] = scrape_followers(main_cast[250:300])
df.iloc[250:300, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
250,A Fall from Grace,Thriller,2020-01-17,120,5.9,English,2020,651070,tt11390036,When a law-abiding woman gets indicted for mur...,...,A Fall from Grace,Drama,Fantasy,United States of America,?,"['Crystal R. Fox', 'Phylicia Rashād', 'Bresha ...",['Tyler Perry'],9,9,"[128k, 298k, 506k, 401k, 6812]"
251,Ultras,Sports film,2020-03-20,108,5.9,Italian,2020,668195,tt10937434,An aging soccer fanatic faces down the reality...,...,Ultras,Drama,?,Italy,?,"['Aniello Arena', 'Simone Borrelli', 'Antonia ...",['Francesco Lettieri'],2,1,"[5150, 12.3k, 6672]"
252,Notes from Dunblane: Lesson from a School Shoo...,Documentary,2018-09-28,23,5.9,English,2018,550860,tt7476418,In the wake of the 2012 Sandy Hook Elementary ...,...,Notes from Dunblane: Lessons from a School Sho...,Documentary,?,United States of America,?,[],['Kim A. Snyder'],0,0,[Create]
253,Friendzone,Romantic comedy,2021-09-29,88,5.9,French,2021,869617,tt14700948,Hopeful romantic Thibault believes his luck co...,...,Friendzone,Romance,Comedy,France,?,"['Mickaël Lumière', 'Manon Azem', 'Éloïse Vall...",['Charles Van Tieghem'],2,2,"[39.7k, 122k, 21.9k, 12.5k]"
254,The Polka King,Comedy-drama,2018-01-12,95,5.9,English,2018,419472,tt5539052,Local Pennsylvania polka legend Jan Lewan deve...,...,The Polka King,Comedy,?,United States of America,?,"['Jack Black', 'Jenny Slate', 'Jason Schwartzm...",['Maya Forbes'],21,8,"[8.5m, 860k, 553, 336, 287k]"
255,The Minimalists: Less Is Now,Documentary,2021-01-01,53,5.9,English,2021,774054,tt13583144,They've built a movement out of minimalism. Lo...,...,The Minimalists: Less Is Now,Documentary,?,United States of America,?,"['Joshua Fields Millburn', 'Ryan Nicodemus']","[""Matt D'Avella""]",2,0,"[95.3k, 78.3k]"
256,Cam,Psychological horror,2018-11-16,94,5.9,English,2018,521935,tt8361028,A young camgirl discovers that she’s inexplica...,...,Cam,Mystery,Thriller,United States of America,?,"['Madeline Brewer', 'Patch Darragh', 'Melora W...",['Daniel Goldhaber'],7,8,"[381k, 13.7k, 2908, 1.9m, 403k]"
257,Maska,Romantic comedy,2020-03-27,111,5.9,Hindi,2020,686919,tt11953628,"Determined to make it in showbiz, an aspiring ...",...,Maska,Comedy,?,India,?,"['Manisha Koirala', 'Prit Kamani', 'Nikita Dut...",['Neeraj Udhwani'],3,3,"[1m, 87.1k, 826k, 177, 7m]"
258,The Decline,Thriller,2020-03-27,83,5.9,French,2020,674607,tt10307440,"Anticipating a disaster, Antoine, a father, at...",...,Jusqu'au déclin,Thriller,?,Canada,?,"['Réal Bossé', 'Marc-André Grondin', 'Guillaum...",['Patrice Laliberté'],5,4,"[94, 36.9k, 2764, 1532, 123]"
259,"My Brother, My Sister",Drama,2021-10-08,110,5.9,Italian,2021,770742,tt13003996,When their father's will forces them to live t...,...,"Mio fratello, mia sorella",Drama,?,Italy,?,"['Alessandro Preziosi', 'Claudia Pandolfi', 'L...",['Roberto Capucci'],2,4,"[80k, 105k, 511k, 31.9k, 58k]"


In [120]:
df.iloc[300:350, -1] = scrape_followers(main_cast[300:350])
df.iloc[340:350, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
340,The Starling,Comedy-drama,2021-09-24,104,6.3,English,2021,468225,tt5164438,A woman adjusting to life after a loss contend...,...,The Starling,Drama,?,United States of America,?,"['Melissa McCarthy', ""Chris O'Dowd"", 'Kevin Kl...",['Theodore Melfi'],8,7,"[10.1m, 1090, 980, 1119, 1m]"
341,Feel the Beat,Family/Comedy-drama,2020-06-19,107,6.3,English,2020,707886,tt10714856,"After failing to make it on Broadway, April re...",...,Feel the Beat,Comedy,Music,Canada,United States of America,"['Sofia Carson', 'Wolfgang Novogratz', 'Donna ...",['Elissa Down'],10,14,"[16.3m, 392k, 53.2k, 14.1k, 18.8k]"
342,Nail Bomber: Manhunt,Documentary,2021-05-26,72,6.3,English,2021,824865,tt12939812,This documentary examines the 1999 London bomb...,...,Nail Bomber: Manhunt,Crime,Documentary,United Kingdom,?,['Tony Blair'],['Daniel Vernon'],1,0,[9402]
343,Notes for My Son,Drama,2020-11-24,83,6.3,Spanish,2020,674865,tt13353486,"Battling terminal cancer, a woman writes a one...",...,El cuaderno de Tomy,Drama,?,Argentina,?,"['Valeria Bertuccelli', 'Esteban Lamothe', 'Ma...",['Carlos Sorín'],5,8,"[6057, 1.2m, Recomendaciones's, 24.7k, 13]"
344,Polar,Action,2019-01-25,118,6.3,English,2019,483906,tt4139588,When a retiring assassin realizes that he is t...,...,Polar,Action,Crime,Germany,United States of America,"['Mads Mikkelsen', 'Vanessa Hudgens', 'Kathery...",['Jonas Åkerlund'],19,22,"[1.2m, 43m, 5.5m, 5305, 371k]"
345,Memories of a Murderer: The Nilsen Tapes,Documentary,2021-08-18,85,6.3,English,2021,852851,tt15085784,Serial killer Dennis Nilsen narrates his life ...,...,Memories of a Murderer: The Nilsen Tapes,Documentary,?,United Kingdom,?,[],['Michael Harte'],0,0,[Create]
346,Spectral,Science fiction/Action,2016-12-09,108,6.3,English,2016,324670,tt2106651,A special-ops team is dispatched to fight supe...,...,Spectral,Science Fiction,Thriller,United States of America,?,"['James Badge Dale', 'Emily Mortimer', 'Gonzal...",['Nic Mathieu'],28,2,"[28.7k, 45.9k, 666, 80.7k, 18.8k]"
347,Shimmer Lake,Crime thriller,2017-06-09,86,6.3,English,2017,457962,tt1386691,Shot in reverse day-by-day through a week—a lo...,...,Shimmer Lake,Thriller,Crime,Canada,United States of America,"['Rainn Wilson', 'Benjamin Walker', 'John Mich...",['Oren Uziel'],11,3,"[2.6m, 75.4k, 4640, 120k, 104k]"
348,The Babysitter,Teen comedy horror,2017-10-13,85,6.3,English,2017,419479,tt4225622,"When Cole stays up past his bedtime, he discov...",...,The Babysitter,Horror,Comedy,United States of America,?,"['Judah Lewis', 'Jimmy Warden', 'Frederick Kee...",['McG'],17,5,"[467k, 5124, 1100, 1.1m, 1.8m]"
349,The Discovery,Science fiction/Drama,2017-03-31,102,6.3,English,2017,376134,tt5155780,"In the near future, due to a breakthrough scie...",...,The Discovery,Drama,Romance,United States of America,?,"['Jason Segel', 'Rooney Mara', 'Robert Redford...",['Charlie McDowell'],9,5,"[269k, 72.9k, 10.8k, 4302, 454k]"


In [122]:
df.iloc[350:400, -1] = scrape_followers(main_cast[350:400])
df.iloc[390:400, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
390,Wheelman,Action thriller,2017-10-20,82,6.4,English,2017,471014,tt5723286,A getaway driver for a bank robbery realizes h...,...,Wheelman,Action,Crime,United States of America,?,"['Frank Grillo', 'Wendy Moniz-Grillo', 'Caitli...",['Jeremy Rush'],6,4,"[571k, 12.8k, 181k, 161k, 1187]"
391,Pray Away,Documentary,2021-08-03,101,6.5,English,2021,685264,tt11224358,"In the 1970s, five men struggling with being g...",...,Pray Away,Documentary,?,United States of America,?,[],['Kristine Stolakis'],0,0,[Create]
392,Dolly Parton: A MusiCares Tribute,Documentary,2021-04-07,55,6.5,English,2021,809208,tt14299926,In a star-studded evening of music and memorie...,...,Dolly Parton: A MusiCares Tribute,Documentary,Music,United States of America,?,"['Dolly Parton', 'Willie Nelson', 'Miley Cyrus...",[],4,6,"[4.5m, 771k, 151m, 2.3m, 140m]"
393,Eurovision Song Contest: The Story of Fire Saga,Musical comedy,2020-06-26,123,6.5,English,2020,531454,tt8580274,Two small-town aspiring musicians chase their ...,...,Eurovision Song Contest: The Story of Fire Saga,Comedy,Music,United States of America,?,"['Rachel McAdams', 'Will Ferrell', 'Pierce Bro...",['David Dobkin'],31,20,"[459k, 116k, 1.7m, 693k, 1844]"
394,A 3 Minute Hug,Documentary,2019-10-28,28,6.5,English/Spanish,2019,617331,tt9098586,As daylight breaks between the border cities o...,...,A 3 Minute Hug,Documentary,?,Mexico,?,[],['Everardo González'],0,0,[Create]
395,All Together Now,Drama,2020-08-28,93,6.5,English,2020,615466,tt3155342,"An optimistic, talented teen clings to a huge ...",...,All Together Now,Drama,?,United States of America,?,"[""Auli'i Cravalho"", 'Rhenzy Feliz', 'Justina M...",['Brett Haley'],3,6,"[120, 78.8k, 352k, 172k, 13.7k]"
396,All the Bright Places,Romance,2020-02-28,108,6.5,English,2020,342470,tt3907584,Two teens facing personal struggles form a pow...,...,All the Bright Places,Romance,Drama,United States of America,?,"['Elle Fanning', 'Justice Smith', 'Alexandra S...",['Brett Haley'],5,7,"[5.3m, 435k, 372k, 103, 275k]"
397,Antoine Griezmann: The Making of a Legend,Documentary,2019-03-21,60,6.5,French,2019,589116,tt9817308,"With heart and determination, Antoine Griezman...",...,Antoine Griezmann: The Making of a Legend,Documentary,?,France,?,"['Antoine Griezmann', 'Didier Deschamps', 'Luc...",['Alex Dell'],4,0,"[35m, 3447, 1.8m, 1.8m]"
398,Chadwick Boseman: Portrait of an Artist,Documentary,2021-04-17,21,6.5,English,2021,818119,tt14476720,An intimate look at the Oscar-nominated actor’...,...,Chadwick Boseman: Portrait of an Artist,Documentary,?,United States of America,?,"['Chadwick Boseman', 'Viola Davis', 'Danai Gur...",['Awol Erizku'],10,4,"[11.2m, 8.3m, 3.2m, 10, 22.2k]"
399,Chopsticks,Comedy,2019-05-31,100,6.5,Hindi,2019,601801,tt10243678,A gifted but insecure woman is in for a transf...,...,Chopsticks,Comedy,Drama,India,?,"['Abhay Deol', 'Mithila Palkar', 'Vijay Raaz',...",['Sachin Yardi'],2,3,"[842k, 3.2m, 2730, 1.1m, 147k]"


In [124]:
df.iloc[400:450, -1] = scrape_followers(main_cast[400:450])
df.iloc[400:450, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
400,Da 5 Bloods,War drama,2020-06-12,155,6.5,English,2020,581859,tt9777644,Four African-American Vietnam veterans return ...,...,Da 5 Bloods,War,Drama,United States of America,?,"['Delroy Lindo', 'Jonathan Majors', 'Clarke Pe...",['Spike Lee'],19,2,"[1437, 20.4k, 55, 47.7k, 2729]"
401,Gerald's Game,Horror thriller,2017-09-29,103,6.5,English,2017,343674,tt3748172,"When her husband's sex game goes wrong, Jessie...",...,Gerald's Game,Horror,Thriller,United States of America,?,"['Carla Gugino', 'Bruce Greenwood', 'Henry Tho...",['Mike Flanagan'],25,12,"[804k, 18.8k, 49.3k, 330k, 446k]"
402,Canvas,Animation / Short,2020-12-11,9,6.5,English,2020,770922,tt9711282,"After a heartbreaking loss, a grandfather stru...",...,Canvas,Animation,Family,United States of America,?,[],['Frank E. Abney III'],0,0,[Create]
403,Altered Carbon: Resleeved,Anime/Science fiction,2020-03-19,74,6.5,Japanese,2020,665251,tt9310328,"On the planet Latimer, Takeshi Kovacs must pro...",...,オルタード・カーボン：リスリーブド,Animation,Science Fiction,Japan,?,"['Tatsuhisa Suzuki', 'Rina Sato', 'Ayaka Asai'...","['Yoshiyuki Okada', 'Takeru Nakajima']",19,10,"[131k, 123, 128, 367, 872k]"
404,Oxygen,Science fiction thriller,2021-05-12,101,6.5,French,2021,471498,tt6341832,A woman wakes in a cryogenic chamber with no r...,...,Oxygène,Science Fiction,Thriller,United States of America,France,"['Mélanie Laurent', 'Mathieu Amalric', 'Malik ...",['Alexandre Aja'],3,3,"[118k, 729, 1191, 39, 40]"
405,Tramps,Romance,2017-04-21,83,6.5,English,2017,412209,tt4991512,A young man and woman find love in an unlikely...,...,Tramps,Comedy,Romance,United States of America,?,"['Callum Turner', 'Grace Van Patten', 'Mike Bi...",['Adam Leon'],3,6,"[11.2k, Photo, 204k, 1506, 1284]"
406,What Did Jack Do?,Drama / Short,2020-01-20,17,6.5,English,2020,528491,tt11644096,"In a locked down train station, a homicide det...",...,WHAT DID JACK DO?,Crime,Comedy,United States of America,France,"['Jack Cruz', 'David Lynch', 'Emily Stofle']",['David Lynch'],2,1,"[41, 292k, 400]"
407,Lust Stories,Drama,2018-06-15,120,6.5,Hindi,2018,530154,tt8439854,An anthology of four stories that sheds light ...,...,Lust Stories,Drama,Romance,India,?,"['Radhika Apte', 'Akash Thosar', 'Randeep Jha'...","['Karan Johar', 'Zoya Akhtar', 'Anurag Kashyap...",6,6,"[3.4m, 501k, 1480, 6.3m, 15.3k]"
408,Murder by the Coast,Documentary,2021-06-23,88,6.5,Spanish,2021,830645,tt13323092,"In 1999, teen Rocío Wanninkhof is murdered. He...",...,El caso Wanninkhof - Carabantes,Documentary,Crime,Spain,?,[],['Tània Balló'],0,0,[Create]
409,Monster,Drama,2021-05-07,98,6.5,English,2021,489932,tt2850272,"The story of Steve Harmon, a 17-year-old honor...",...,Monster,Crime,Drama,Canada,United States of America,"['Kelvin Harrison Jr.', 'Jeffrey Wright', 'Jen...",['Anthony Mandler'],14,5,"[74.9k, 112k, 3.3m, 192, 15.2k]"


In [126]:
df.iloc[450:500, -1] = scrape_followers(main_cast[450:500])
df.iloc[490:500, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
490,7 años,Drama,2016-10-28,76,6.8,Spanish,2016,423093,tt5517438,The drama is centered on four friends and busi...,...,7 años,Drama,?,Spain,?,"['Paco León', 'Juan Pablo Raba', 'Alex Brendem...",['Roger Gual'],3,2,"[1.8m, 908k, 3842, 468k, 495k]"
491,Blood Brothers: Malcolm X & Muhammad Ali,Documentary,2021-09-09,96,6.8,English,2021,861604,tt15095938,"From a chance meeting to a tragic fallout, Mal...",...,Blood Brothers: Malcolm X and Muhammad Ali,Documentary,History,United States of America,?,"['Malcolm X', 'Muhammad Ali', 'Al Sharpton', '...",['Marcus A. Clarke'],7,2,"[35k, 4.7m, Rev., 157k, 98.1k]"
492,A Life of Speed: The Juan Manuel Fangio Story,Documentary,2020-03-20,92,6.8,Spanish,2020,675598,tt6668212,"Juan Manuel Fangio was the Formula One king, w...",...,"Fangio, el hombre que domaba las máquinas",Documentary,?,Argentina,?,"['Fernando Alonso', 'Alain Prost', 'Jackie Ste...",['Francisco Macri'],5,0,"[3.6m, 132k, 70.4k, Two-time, 1.7m]"
493,A Love Song for Latasha,Documentary,2020-09-21,19,6.8,English,2020,631344,tt8993180,A dreamlike conversation with the past and the...,...,A Love Song for Latasha,Documentary,?,United States of America,?,['Brittany Hudson'],['Sophia Nahli Allison'],0,1,[brittany.hudson.]
494,All in My Family,Documentary,2019-05-03,39,6.8,English/Mandarin,2019,595399,tt9097148,After starting a family of his very own in the...,...,All in My Family,Documentary,?,China,United States of America,[],['Hao Wu'],0,0,[Create]
495,Always Be My Maybe,Romantic comedy,2019-05-31,102,6.8,English,2019,513576,tt7374948,"Reunited after 15 years, famous chef Sasha and...",...,Always Be My Maybe,Romance,Comedy,United States of America,?,"['Ali Wong', 'Randall Park', 'James Saito', 'M...",['Nahnatchka Khan'],16,14,"[1.9m, 13.1k, 3817, 236k, 5945]"
496,Becoming,Documentary,2020-05-06,89,6.8,English,2020,699280,tt12221748,Join former first lady Michelle Obama in an in...,...,Becoming,Documentary,?,United States of America,?,"['Michelle Obama', 'Barack Obama', 'Phoebe Rob...",['Nadia Hallgren'],3,7,"[47.6m, 34.9m, 349k, 232k, 232k]"
497,A Futile and Stupid Gesture,Biographical/Comedy,2018-01-26,101,6.8,English,2018,391710,tt5566790,The National Lampoon name became globally reco...,...,A Futile and Stupid Gesture,Comedy,?,United States of America,?,"['Will Forte', 'Domhnall Gleeson', 'Martin Mul...",['David Wain'],36,16,"[77.7k, 14.4k, 28, 170k, 527k]"
498,The Meyerowitz Stories (New and Selected),Comedy-drama,2017-10-13,112,6.9,English,2017,396398,tt5536736,An estranged family gathers together in New Yo...,...,The Meyerowitz Stories (New and Selected),Comedy,Drama,United States of America,?,"['Adam Sandler', 'Ben Stiller', 'Dustin Hoffma...",['Noah Baumbach'],17,15,"[11.8m, 1.7m, 2854, 11.7k, 141]"
499,Mank,Biopic,2020-12-04,132,6.9,English,2020,614560,tt10618286,1930s Hollywood is reevaluated through the eye...,...,Mank,Drama,History,United States of America,?,"['Gary Oldman', 'Amanda Seyfried', 'Lily Colli...",['David Fincher'],34,17,"[136k, 5.1m, 24m, 143, 108k]"


In [129]:
df.iloc[500:550, -1] = scrape_followers(main_cast[500:550])
df.iloc[500:550, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
500,Cops and Robbers,Animation / Short,2020-12-28,7,6.9,English,2020,776979,tt13322484,Animation and activism unite in this multimedi...,...,Cops and Robbers,Animation,?,United States of America,?,['Timothy Ware-Hill'],"['Arnon Manor', 'Timothy Ware-Hill']",1,0,[3245]
501,The Highwaymen,Crime drama,2019-03-29,131,6.9,English,2019,500682,tt1860242,"In 1934, Frank Hamer and Manny Gault, two form...",...,The Highwaymen,Crime,Drama,United States of America,?,"['Kevin Costner', 'Woody Harrelson', 'Kathy Ba...",['John Lee Hancock'],41,6,"[654k, 2.3m, 164k, 28.4k, 107k]"
502,AK vs AK,Thriller,2020-12-24,108,6.9,Hindi,2020,735919,tt11651796,"After a public spat with a movie star, a disg...",...,AK vs AK,Comedy,Drama,India,?,"['Anil Kapoor', 'Anurag Kashyap', 'Sonam Kapoo...",['Vikramaditya Motwane'],5,1,"[4.2m, 1m, 31.5m, 702k, 756]"
503,Amanda Knox,Documentary,2016-09-30,92,6.9,English,2016,411009,tt5952332,"This gripping, atmospheric documentary recount...",...,Amanda Knox,Documentary,Crime,Denmark,United States of America,"['Amanda Knox', 'Stefano Conti', 'Anderson Coo...","['Rod Blackhurst', 'Brian McGinn']",4,1,"[100k, 548, 3m, 23.7m, 130]"
504,Biggie: I Got a Story to Tell,Documentary,2021-03-01,97,6.9,English,2021,797309,tt14058484,"Christopher Wallace, AKA The Notorious B.I.G.,...",...,Biggie: I Got a Story to Tell,Documentary,Music,United States of America,?,"['The Notorious B.I.G.', 'Sean Combs', 'Faith ...",['Emmett Malloy'],21,6,"[956k, 18.3m, 2.1m, 88.7k, 120]"
505,I Don't Feel at Home in This World Anymore,Drama,2017-02-24,96,6.9,English,2017,425591,tt5710514,"When a depressed woman is burglarized, she fin...",...,I Don't Feel at Home in This World Anymore,Comedy,Crime,United States of America,?,"['Melanie Lynskey', 'Elijah Wood', 'David Yow'...",['Macon Blair'],15,13,"[29.6k, 24.2k, Helmet, 298k, 134]"
506,Laerte-se,Documentary,2017-05-19,100,6.9,Portuguese,2017,450628,tt6791240,"In this film, Laerte conjugates the body in th...",...,Laerte-se,Documentary,?,Brazil,?,"['Laerte Coutinho', 'Eliane Brum', 'Rita Lee',...","['Eliane Brum', 'Lygia Barbosa da Silva']",0,4,"[134k, 59.8k, 751k, 683k]"
507,Bigflo & Oil: Hip Hop Frenzy,Documentary,2020-10-08,100,6.9,French,2020,746752,tt13161318,Go backstage with French rap duo Bigflo & Oli ...,...,Bigflo & Oli : Presque Trop,Documentary,Music,France,?,"['Bigflo', 'Oli']","['Oli', 'Bigflo']",2,0,"[2.1m, 483k]"
508,Haseen Dillruba,Romantic thriller,2021-07-02,135,6.9,Hindi,2021,695969,tt11027830,Under investigation as a suspect in her husban...,...,हसीन दिलरुबा,Mystery,Crime,India,?,"['Taapsee Pannu', 'Vikrant Massey', 'Harshvard...",['Vinil Mathew'],3,2,"[19.3m, 1.1m, 1.9m, 90.9k, 752]"
509,Pagglait,Comedy-drama,2021-03-26,114,6.9,Hindi,2021,786190,tt11142762,"Widowed soon after marriage, a young woman gra...",...,पगलैट,Drama,Comedy,India,?,"['Sanya Malhotra', 'Shruti Sharma', 'Ashutosh ...",['Umesh Bist'],3,7,"[2.2m, 1m, 59.2k, 11.7k, 6978]"


In [131]:
df.iloc[550:600, -1] = scrape_followers(main_cast[550:600])
df.iloc[590:600, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
590,Get Me Roger Stone,Documentary,2017-05-12,101,7.3,English,2017,448448,tt6714534,From his days of testifying at the Watergate h...,...,Get Me Roger Stone,Documentary,History,United States of America,?,"['Roger Stone', 'Donald Trump', 'Jeffrey Toobi...","['Dylan Bank', 'Morgan Pehme', 'Daniel DiMauro']",29,7,"[3376, 23.7m, 1907, 985, 325k]"
591,Father Soldier Son,Documentary,2020-07-17,100,7.3,English,2020,695476,tt12187586,When Sgt. First Class Brian Eisch is criticall...,...,Father Soldier Son,Documentary,War,United States of America,?,[],"['Leslye Davis', 'Catrin Einhorn']",0,0,[Create]
592,Extremis,Documentary,2016-09-13,24,7.3,English,2016,393443,tt5538078,A purely observational non-fiction film that t...,...,Extremis,Documentary,?,United States of America,?,[],['Dan Krauss'],0,0,[Create]
593,The Fundamentals of Caring,Comedy-drama,2016-06-24,97,7.3,English,2016,318121,tt2452386,"Having suffered a tragedy, Ben becomes a careg...",...,The Fundamentals of Caring,Comedy,Drama,United States of America,?,"['Paul Rudd', 'Craig Roberts', 'Selena Gomez',...",['Rob Burnett'],10,8,"[77k, 76.2k, 272m, 192, 11.5k]"
594,Okja,Action-adventure,2017-06-28,121,7.3,English/Korean,2017,387426,tt3967856,A young girl named Mija risks everything to pr...,...,Okja,Adventure,Drama,South Korea,United States of America,"['Ahn Seo-hyun', 'Tilda Swinton', 'Paul Dano',...",['Bong Joon-ho'],33,16,"[94k, 57.2k, 7689, 7m, 24m]"
595,Dolemite Is My Name,Biopic,2019-10-25,118,7.3,English,2019,528888,tt8526872,"The story of Rudy Ray Moore, who created the i...",...,Dolemite Is My Name,Drama,Comedy,United States of America,?,"['Eddie Murphy', 'Keegan-Michael Key', 'Mike E...",['Craig Brewer'],33,8,"[19.1k, 949k, 7.1m, 292k, 523k]"
596,Shiny_Flakes: The Teenage Drug Lord,Documentary,2021-08-03,96,7.3,German,2021,850329,tt15036416,Max S. reveals how he built a drug empire from...,...,Shiny_Flakes: The Teenage Drug Lord,Documentary,Crime,?,?,['Maximilian Schmidt'],"['Michael Schmitt', 'Eva Müller']",1,0,[4212]
597,Found,Documentary,2021-10-20,98,7.3,English,2021,819500,tt13622146,Follows the story of three American teenage gi...,...,Found,Documentary,?,China,United States of America,[],['Amanda Lipitz'],0,0,[Create]
598,I'm No Longer Here,Drama,2020-05-27,105,7.3,Spanish,2020,582927,tt4323594,"In Monterrey, Mexico, a young street gang spen...",...,Ya no estoy aquí,Drama,?,Mexico,United States of America,['Juan Daniel Garcia Treviño'],['Fernando Frias'],1,0,[107k]
599,El Camino: A Breaking Bad Movie,Crime drama,2019-10-11,121,7.3,English,2019,559969,tt9243946,In the wake of his dramatic escape from captiv...,...,El Camino: A Breaking Bad Movie,Crime,Drama,United States of America,?,"['Aaron Paul', 'Jesse Plemons', 'Charles Baker...",['Vince Gilligan'],19,6,"[5m, 4302, 48.3k, 451k, 16.3k]"


In [133]:
df.iloc[600:, -1] = scrape_followers(main_cast[600:])
df.iloc[600:, :]

C:\Users\Santiago\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\Santiago\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,Title,Genre,Premiere,Runtime,IMDB Score,Language,premiere_year,tmdb_id,imdb_id,overview,...,original_title,genre_1,genre_2,country_1,country_2,cast,director,male_actors,female_actors,followers
600,Mucho Mucho Amor: The Legend of Walter Mercado,Documentary,2020-07-08,96,7.3,Spanish/English,2020,653735,tt11378264,"Once the world's most famous astrologer, Walte...",...,Mucho Mucho Amor: The Legend of Walter Mercado,Documentary,?,United States of America,?,"['Walter Mercado', 'Lin-Manuel Miranda', 'Raul...","['Kareem Tabsch', 'Cristina Costantini']",4,1,"[46.5k, 80.3k, 1.5m, 17.3m, 4973]"
601,Sitara: Let Girls Dream,Animation / Short,2020-03-08,15,7.3,English,2020,651166,tt11064862,"Pari, a 14-year-old girl, dreams of becoming a...",...,Sitara,Animation,?,Pakistan,United States of America,[],['Sharmeen Obaid-Chinoy'],0,0,[Create]
602,The Other One: The Long Strange Trip of Bob Weir,Documentary,2015-05-22,83,7.3,English,2015,261039,tt3692768,Drop out of school to ride with the Merry Pran...,...,"The Other One: The Long, Strange Trip of Bob Weir",Documentary,Music,?,?,['Bob Weir'],['Mike Fleiss'],1,0,[365k]
603,Octonauts & the Great Barrier Reef,Animation,2020-10-13,47,7.3,English,2020,752885,tt13150606,"In this musical special, the Octonauts must fi...",...,Octonauts and the Great Barrier Reef,Family,Music,?,?,"['Simon Greenall', 'Rob Rackstraw', 'Jo Wyatt'...",['Blair Simmons'],3,1,"[556, 540, 220, 11k]"
604,The Death and Life of Marsha P. Johnson,Documentary,2017-10-06,105,7.3,English,2017,448444,tt5233558,"Describing herself as a 'street queen,' Johnso...",...,The Death and Life of Marsha P. Johnson,Documentary,?,United States of America,?,"['Taylor Mead', 'Vito Russo']",['David France'],2,0,"[3158, 663]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,Taylor Swift: Reputation Stadium Tour,Concert Film,2018-12-31,125,8.4,English,2018,568332,tt9426852,Taylor Swift takes the stage in Dallas for the...,...,Taylor Swift: Reputation Stadium Tour,Music,Documentary,United States of America,?,"['Taylor Swift', 'David Cook', 'Amos Heller', ...",['Paul Dugdale'],2,5,"[184m, 55.7k, 43.2k, 64.6k, 3.8m]"
680,Winter on Fire: Ukraine's Fight for Freedom,Documentary,2015-10-09,91,8.4,English/Ukranian/Russian,2015,355020,tt4908644,A documentary on the unrest in Ukraine during ...,...,Winter on Fire: Ukraine's Fight for Freedom,Documentary,?,United Kingdom,United States of America,['Catherine Ashton'],['Evgeny Afineevsky'],0,1,[1724]
681,Springsteen on Broadway,One-man show,2018-12-16,153,8.5,English,2018,563708,tt8716334,Springsteen on Broadway is a solo acoustic per...,...,Springsteen On Broadway,Music,Documentary,?,?,['Bruce Springsteen'],['Thom Zimny'],1,0,[1.2m]
682,Emicida: AmarElo - It's All For Yesterday,Documentary,2020-12-08,89,8.6,Portuguese,2020,765613,tt13458600,Between scenes from his concert in São Paulo's...,...,Emicida: AmarElo - É Tudo Pra Ontem,Documentary,Music,Brazil,?,"['Emicida', 'Fernanda Montenegro', 'Zeca Pagod...",['Fred Ouro Preto'],6,4,"[2.4m, 650k, 2.4m, 12m, 42.3k]"


In [134]:
# Save the dataframe as checkpoint
df.to_excel("temporary_netflix_df.xlsx")